In [1]:
import importlib
import numpy as np
import math as m
import statistics as st
import cmath
import matplotlib.pyplot as plt 
import itertools
import random
from operator import add
from graphviz import Source, render
import pydot

import datetime
import time

import sys
sys.path.append('py_modules') # below modules are in this folder
#print(sys.path)
import setup_nx # your own module, setup.nx.py
importlib.reload(setup_nx)
import my_feeder_funcs as ff
import my_impedance_funcs as imp
import my_configVis_funcs as vis
import my_detControlMatExistence_funcs as ctrl
import my_detLznRange_funcs as lzn
import my_heatmapSetup_funcs as hm

In [2]:
# [Essential] specify input feeder data for IEEE 123-node test feeder

# Enter the path/name of the impedance model data (excel file)
filepath = "feeder_impedance_models/"
#modelpath = filepath + "003_GB_IEEE37.xlsx" # accurate wrt public 123NF data
modelpath = filepath + "003_GB_IEEE37_zeroCrossZ.xlsx" # accurate wrt public 123NF data

#file_name = string specifying name of dot file created when make_graph() is called
file_name = '37NF'
#file_name = 'demo123NF.dot'

# Specify substation kV, kVA bases, name, and the number of timesteps in the load data'
Vbase_ll = 4800
Vbase = Vbase_ll / np.sqrt(3)
Sbase = 5000/3
substation_name = 'bus_799'
timesteps = 1

# initialize some variables
ts = time.time()
print(datetime.datetime.fromtimestamp(ts))
plot = 0 #turn plot on/off
depths = {}
leaves = []

2022-05-26 15:20:12.135698


In [3]:
# [ESSENTIAL] create feeder object

#fin_feeder = ff.feeder_init(modelpath,loadfolder,loadpath,timesteps,Vbase_ll,Sbase,depths,leaves)
fin_feeder = ff.feeder_init(modelpath,'','',timesteps,Vbase_ll,Sbase,depths,leaves)

print("Finished initializing feeder")
ff.make_graph(fin_feeder, file_name)
node_index_map = hm.createNodeIndexMap(fin_feeder) #node indices for indicMat and F matrix
print(list(node_index_map.items())[:10]) # print first 10 elements of node_index_map

R,X=hm.createRXmatrices_3ph(fin_feeder, node_index_map,depths,file_name)
n=round(len(R)/3)
print('n=',n)

# #print('depths=',depths) # check this is populated, lists how far each node is from substation
# #print('depths length=',len(depths))



Finished initializing feeder
[('bus_701', 0), ('bus_702', 1), ('bus_703', 2), ('bus_704', 3), ('bus_705', 4), ('bus_706', 5), ('bus_707', 6), ('bus_708', 7), ('bus_709', 8), ('bus_710', 9)]
graph_noSub= ['bus_702' 'bus_703' 'bus_704' 'bus_705' 'bus_706' 'bus_707' 'bus_708'
 'bus_709' 'bus_710' 'bus_711' 'bus_712' 'bus_713' 'bus_714' 'bus_718'
 'bus_720' 'bus_722' 'bus_724' 'bus_725' 'bus_727' 'bus_728' 'bus_729'
 'bus_730' 'bus_731' 'bus_732' 'bus_733' 'bus_734' 'bus_735' 'bus_736'
 'bus_737' 'bus_738' 'bus_740' 'bus_741' 'bus_742' 'bus_744' 'bus_775']
n= 35


In [ ]:
print(node_index_map)

In [4]:
# Save R and X matrices to csv to import into matlab
# np.savetxt reference: https://thispointer.com/how-to-save-numpy-array-to-a-csv-file-using-numpy-savetxt-in-python/
np.savetxt('Rmat_37NF_mod.csv', R, delimiter=',')
np.savetxt('Xmat_37NF_mod.csv', X, delimiter=',')

In [6]:
# write busnames into a csv
import csv
graphNodes_noSub=hm.remove_subst_nodes(fin_feeder, file_name) # remove substation nodes, busList will have as many entries as R and X matrix length
assert(len(graphNodes_noSub)*3==len(R)) # *3 is because R is 3ph
with open("37NF_busList.csv", 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter='-',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerows(graphNodes_noSub)